# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,anadyr,64.7500,177.4833,262.43,92,23,2.00,RU,1728769877
1,1,waitangi,-43.9535,-176.5597,284.72,99,100,3.13,NZ,1728769901
2,2,ushuaia,-54.8000,-68.3000,281.96,49,40,6.69,AR,1728769902
3,3,yellowknife,62.4560,-114.3525,280.60,87,23,2.24,CA,1728769903
4,4,capitan bado,-23.2667,-55.5333,295.80,64,31,3.27,PY,1728769904


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
map_plot = city_data_df.hvplot.points( "Lng", "Lat", geo = True, tiles = "OSM", frame_width = 800, frame_height = 500, hover_cols=['City', 'Humidity'], size = "Humidity", alpha=0.6, scale = 0.2, color = "City")
map_plot

C:\Users\ibraa\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
criteria = city_data_df[(city_data_df['Max Temp'] > 15) & (city_data_df['Cloudiness'] > 50) & (city_data_df['Wind Speed'] > 5)]

# Drop any rows with null values
criteria = criteria.dropna()

# Display sample data
print(criteria)

     City_ID                 City      Lat       Lng  Max Temp  Humidity  \
5          5    port-aux-francais -49.3500   70.2167    276.26        83   
6          6         punta arenas -53.1500  -70.9167    283.21        62   
9          9          nar'yan-mar  67.6713   53.0870    275.30        85   
14        14            salekhard  66.5300   66.6019    273.94        72   
16        16             la ronge  55.1001 -105.2842    282.27        66   
..       ...                  ...      ...       ...       ...       ...   
546      546  san antonio de pale  -1.4014    5.6325    298.77        80   
552      552             klaksvik  62.2266   -6.5890    277.67        65   
562      562         fuglafjordur  62.2440   -6.8140    276.53        67   
569      569          goondiwindi -28.5500  150.3167    292.17        55   
578      578              kragero  58.8693    9.4149    283.65        73   

     Cloudiness  Wind Speed Country        Date  
5            97       10.07      TF  

### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head() 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,anadyr,RU,64.7500,177.4833,92,
1,waitangi,NZ,-43.9535,-176.5597,99,
2,ushuaia,AR,-54.8000,-68.3000,49,
3,yellowknife,CA,62.4560,-114.3525,87,
4,capitan bado,PY,-23.2667,-55.5333,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}
# Print a message to follow up the hotel search
print("Starting hotel search")
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    # Make and API request using the params dictionary
    name_address = response = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address = response.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
# Display sample data
hotel_df

Starting hotel search
anadyr - nearest hotel: Гостевой дом
waitangi - nearest hotel: Hotel Chathams
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
yellowknife - nearest hotel: No hotel found
capitan bado - nearest hotel: Hotel Esmeralda
port-aux-francais - nearest hotel: Keravel
punta arenas - nearest hotel: Hotel Lacolet
pennagaram - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
nar'yan-mar - nearest hotel: Гостиница "Агат"
balkanabat - nearest hotel: Nebitçi
molchanovo - nearest hotel: Уют
margaret river - nearest hotel: Margaret River Hotel
aasiaat - nearest hotel: SØMA
salekhard - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
la ronge - nearest hotel: La Ronge Hotel and Suites
darwin - nearest hotel: Mantra Pantanas Darwin
isafjordur - nearest hotel: Hótel Horn
papatowai - nearest hotel: No hotel found
smithers - nearest hotel: Sunshine Inn Hotel
machico - nearest hotel: Residence Inn
trakai - nearest hotel: Prie žiedo
lerw

,City,Country,Lat,Lng,Humidity,Hotel Name
0,anadyr,RU,64.7500,177.4833,92,Гостевой дом
1,waitangi,NZ,-43.9535,-176.5597,99,Hotel Chathams
2,ushuaia,AR,-54.8000,-68.3000,49,Apart Hotel Aires del Beagle
3,yellowknife,CA,62.4560,-114.3525,87,No hotel found
4,capitan bado,PY,-23.2667,-55.5333,64,Hotel Esmeralda
...,...,...,...,...,...,...
574,comodoro rivadavia,AR,-45.8667,-67.5000,47,No hotel found
575,bousso,TD,10.4795,16.7117,87,No hotel found
576,baymak,RU,52.5933,58.3225,84,No hotel found
577,soni,TZ,-4.8500,38.3667,91,Maweni Farm Lodge


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", "Lat", 
    geo=True, 
    tiles="OSM", 
    frame_width=800, 
    frame_height=500, 
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'], 
    size="Humidity", 
    color="Hotel Name", 
    alpha=0.6,
    scale=0.2
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity,City,Country)